In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge
import itertools
from scipy.integrate import odeint
from sklearn.preprocessing import StandardScaler
import json
from tqdm import tqdm

In [3]:
def compute_valid_prediction_time(y_true, y_pred, t_vals, threshold, lambda_max, dt):
    """
    Compute the Valid Prediction Time (VPT) and compare it to Lyapunov time T_lambda = 1 / lambda_max.
    
    Parameters
    ----------
    y_true : ndarray of shape (N, dim)
        True trajectory over time.
    y_pred : ndarray of shape (N, dim)
        Model's predicted trajectory over time (closed-loop).
    t_vals : ndarray of shape (N,)
        Time values corresponding to the trajectory steps.
    threshold : float, optional
        The error threshold, default is 0.4 as in your snippet.
    lambda_max : float, optional
        Largest Lyapunov exponent. Default=0.9 for Lorenz.
        
    Returns
    -------
    T_VPT : float
        Valid prediction time. The earliest time at which normalized error surpasses threshold
        (or the last time if never surpassed).
    T_lambda : float
        Lyapunov time = 1 / lambda_max
    ratio : float
        How many Lyapunov times the model prediction remains valid, i.e. T_VPT / T_lambda.
    """
    # 1) Average of y_true
    y_mean = np.mean(y_true, axis=0)  # shape (dim,)
    
    # 2) Time-averaged norm^2 of (y_true - y_mean)
    y_centered = y_true - y_mean
    denom = np.mean(np.sum(y_centered**2, axis=1))  # scalar
    
    # 3) Compute the normalized error delta_gamma(t) = ||y_true - y_pred||^2 / denom
    diff = y_true - y_pred
    err_sq = np.sum(diff**2, axis=1)  # shape (N,)
    delta_gamma = err_sq / denom      # shape (N,)
    
    # 4) Find the first time index where delta_gamma(t) exceeds threshold
    idx_exceed = np.where(delta_gamma > threshold)[0]
    if len(idx_exceed) == 0:
        # never exceeds threshold => set T_VPT to the final time
        T_VPT = t_vals[-1]
    else:
        T_VPT = t_vals[idx_exceed[0]]
    
    # 5) Compute T_lambda and ratio
    T_lambda = 1.0 / lambda_max

    # print(f"\n--- Valid Prediction Time (VPT) with threshold={threshold}, lambda_max={lambda_max} ---")

    T_VPT = (T_VPT - t_vals[0])  # Adjust T_VPT to be relative to the start time
    ratio = T_VPT / T_lambda

    return T_VPT, T_lambda, ratio

In [4]:
def compute_attractor_deviation(predictions, targets, cube_size=(0.1, 0.1, 0.1)):
    """
    Compute the Attractor Deviation (ADev) metric.

    Parameters:
        predictions (numpy.ndarray): Predicted trajectories of shape (n, 3).
        targets (numpy.ndarray): True trajectories of shape (n, 3).
        cube_size (tuple): Dimensions of the cube (dx, dy, dz).

    Returns:
        float: The ADev metric.
    """
    # Define the cube grid based on the range of the data and cube size
    min_coords = np.min(np.vstack((predictions, targets)), axis=0)
    max_coords = np.max(np.vstack((predictions, targets)), axis=0)

    # Create a grid of cubes
    grid_shape = ((max_coords - min_coords) / cube_size).astype(int) + 1

    # Initialize the cube occupancy arrays
    pred_cubes = np.zeros(grid_shape, dtype=int)
    target_cubes = np.zeros(grid_shape, dtype=int)

    # Map trajectories to cubes
    pred_indices = ((predictions - min_coords) / cube_size).astype(int)
    target_indices = ((targets - min_coords) / cube_size).astype(int)

    # Mark cubes visited by predictions and targets
    for idx in pred_indices:
        pred_cubes[tuple(idx)] = 1
    for idx in target_indices:
        target_cubes[tuple(idx)] = 1

    # Compute the ADev metric
    adev = np.sum(np.abs(pred_cubes - target_cubes))
    return adev

In [5]:
def nrmse_dimwise(pred, truth):
    length = min(len(pred), len(truth))
    pred = pred[:length]
    truth = truth[:length]
    mse = np.mean((pred - truth) ** 2, axis=0)
    std = np.std(truth, axis=0)
    std[std == 0] = 1e-8
    nrmse = np.sqrt(mse) / std
    return np.linalg.norm(nrmse)

In [6]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

In [7]:
def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

In [ ]:
class MCI3D:
    """
    Minimum Complexity Interaction ESN (MCI-ESN).

    This class implements the approach described in:
      "A Minimum Complexity Interaction Echo State Network"
        by Jianming Liu, Xu Xu, Eric Li (2024).
    
    The model structure:
      - We maintain two 'simple cycle' reservoirs (each of size N).
      - Each reservoir is a ring with weight = l, i.e. 
            W_res[i, (i+1)%N] = l
        plus the corner wrap from (N-1)->0, also = l. ##(unnecessary as already called for in the prev. line)
      - The two reservoirs interact via a minimal connection matrix: 
         exactly 2 cross-connections with weight = g. 
         (One might connect x2[-1], x2[-2], ... 
          But we do where reservoir1 sees x2[-1] 
          in one location, and reservoir2 sees x1[-1] likewise.)
      - Activation function in reservoir1 is cos(·), and in reservoir2 is sin(·).
      - They each have a separate input weight matrix: Win1 and Win2. 
        The final state is a linear combination 
           x(t) = h*x1(t) + (1-h)*x2(t).
      - Then we do a polynomial readout [x, x^2, 1] -> output.
      - We feed teacher forcing in collect_states, 
        then solve readout with Ridge regression.

    References:
      - Liu, J., Xu, X., & Li, E. (2024). 
        "A minimum complexity interaction echo state network," 
         Neural Computing and Applications.
    
    notes:
      - The reservoir_size is N for each reservoir, 
        so total param dimension is 2*N for states, 
        but we produce a single final "combined" state x(t) in R^N for readout.
      - The activation f1=cos(...) for reservoir1, f2=sin(...) for reservoir2, 
        as recommended by the paper for MCI-ESN.

    """

    def __init__(
        self,
        reservoir_size=500,
        cycle_weight=0.9,      # 'l' in the paper
        connect_weight=0.9,    # 'g' in the paper
        input_scale=0.2,
        leaking_rate=1.0,
        ridge_alpha=1e-6,
        combine_factor=0.1,    # 'h' in the paper
        seed=47,
        v1=0.6, v2=0.6         # fixed values for v1, v2
    ):
        """
        reservoir_size: N, size of each cycle reservoir 
        cycle_weight : l, ring adjacency weight in [0,1), ensures cycle synergy
        connect_weight: g, cross-connection weight between the two cycle reservoirs
        input_scale   : scale factor for input->reservoir weights
        leaking_rate  : ESN update alpha 
        ridge_alpha   : readout ridge penalty
        combine_factor: h in [0,1], to form x(t)= h*x1(t)+(1-h)*x2(t) as final combined state
        seed          : random seed
        """
        self.reservoir_size = reservoir_size
        self.cycle_weight   = cycle_weight
        self.connect_weight = connect_weight
        self.input_scale    = input_scale
        self.leaking_rate   = leaking_rate
        self.ridge_alpha    = ridge_alpha
        self.combine_factor = combine_factor
        self.seed           = seed
        self.v1 = v1
        self.v2 = v2

        # We'll define (and build) adjacency for each cycle, 
        # plus cross-connection for two sub-reservoirs.
        # We'll define 2 input weight mats: Win1, Win2.
        # We'll define states x1(t), x2(t).
        # We'll define readout W_out after training.

        self._build_mci_esn()

    def _build_mci_esn(self):
        """
        Build all the internal parameters: 
         - ring adjacency for each reservoir
         - cross-reservoir connection
         - input weights for each reservoir
         - initial states
        """
        np.random.seed(self.seed)

        N = self.reservoir_size

        # Build ring adjacency W_res in shape [N, N], with cycle_weight on ring
        W_res = np.zeros((N, N))
        for i in range(N):
            j = (i+1) % N
            W_res[j, i] = self.cycle_weight
        self.W_res = W_res  # shared by both sub-reservoirs

        # Build cross-connection W_cn for shape [N,N], 
        # minimal 2 nonzero elements. 
        # For the simplest approach from the paper:
        #   W_cn[0, N-1] = g, W_cn[1, N-2] = g or similar.
        # The paper's eq(7) suggests the last 2 elements in x(t) cross to first 2 in the other reservoir:
        # We'll do the simplest reference: if i=0 or i=1, we connect from the other reservoir's last or second-last. 
        # We'll define a function for each sub-res to pick up from the other sub-res. 
        # We can store them in separate arrays, or define them in code. 
        # We'll just store "We want index 0 to see x2[-1], index 1 to see x2[-2]."

        # But as done in the original code snippet from the paper:
        #   Wcn has
        # effectively 2 nonzero positions. We'll define that pattern:
        W_cn = np.zeros((N, N))
        # e.g. W_cn[0, N-1] = g, W_cn[N-1, N-2] = g or something. 
        # The paper example used W_cn = diag(0,g,...) plus the corner. We'll do the simplest:
        # let W_cn[0, N-1]=g, W_cn[1, N-2]=g.
        # This matches the minimal cross. 
        # For clarity we do:
        W_cn[0, N-1] = self.connect_weight
        if N>1:
            # W_cn[1, N-2] = self.connect_weight
            W_cn[N-1, 0] = self.connect_weight
        self.W_cn = W_cn

        # We'll define input weights for each sub-reservoir, shape [N, dim_input].
        # The paper sets them as eq(10) in the snippet, with different signs. 
        # We'll define them as parted. 
        # We define V1, V2 => shape [N, dim_input], with constant magnitude t1, t2, random sign. 
        # We'll do random. Need to check this in the paper again
        # We'll keep "two" separate. user can define input_scale but not two separate. 
        # We'll do the simplest approach: the absolute value is the same => input_scale, 
        # sign is random. Then we define Win1 = V1 - V2, Win2 = V1 + V2.
        # This is consistent with eq(10) from the paper.

        self.Win1 = None
        self.Win2 = None

        # We'll define states x1(t), x2(t). We'll do them after dimension known. 
        self.x1 = None
        self.x2 = None

        self.W_out = None

    def _init_substates(self):
        """
        Once we know reservoir_size, we define x1, x2 as zeros. 
        We'll call this in reset_state or at fit time.
        """
        N = self.reservoir_size
        self.x1 = np.zeros(N)
        self.x2 = np.zeros(N)

    def reset_state(self):
        if self.x1 is not None:
            self.x1[:] = 0.0
        if self.x2 is not None:
            self.x2[:] = 0.0

    def _update(self, u):
        """
        Single-step reservoir update.
        x1(t+1) = cos( Win1*u(t+1) + W_res*x1(t) + W_cn*x2(t) )
        x2(t+1) = sin( Win2*u(t+1) + W_res*x2(t) + W_cn*x1(t) )
        Then x(t)= h*x1(t+1) + (1-h)* x2(t+1).
        We'll define the leaky integration. 
        But the paper uses an approach with no leak? Be careful.
        We'll do the approach: x1(t+1)= (1-alpha)* x1(t) + alpha*cos(...).
        """
        alpha = self.leaking_rate

        # pre activation for reservoir1
        pre1 = self.Win1 @ u + self.W_res @ self.x1 + self.W_cn @ self.x2
        # reservoir1 uses cos
        new_x1 = np.cos(pre1)

        # reservoir2 uses sin
        pre2 = self.Win2 @ u + self.W_res @ self.x2 + self.W_cn @ self.x1
        new_x2 = np.sin(pre2)

        self.x1 = (1.0 - alpha)*self.x1 + alpha*new_x1
        self.x2 = (1.0 - alpha)*self.x2 + alpha*new_x2

    def _combine_state(self):
        """
        Combine x1(t), x2(t) => x(t) = h*x1 + (1-h)*x2
        """
        h = self.combine_factor
        return h*self.x1 + (1.0 - h)*self.x2

    def collect_states(self, inputs, discard=100):
        # We reset the reservoir to zero
        self.reset_state()
        states = []
        for t in range(len(inputs)):
            self._update(inputs[t])   # feed the REAL input from the dataset
            combined = self._combine_state()
            states.append(combined.copy())
        states = np.array(states)  # shape => [T, N]
        return states[discard:], states[:discard]


    def fit_readout(self, train_input, train_target, discard=100):
        """
        Build input weights if needed, gather states on the training data (teacher forcing),
        then solve a polynomial readout [x, x^2, 1]->train_target(t).

        train_input : shape [T, d_in]
        train_target: shape [T, d_out]
        discard     : # of states to discard for warmup
        """
        T = len(train_input)
        if T<2:
            raise ValueError("Not enough training data")

        d_in = train_input.shape[1]
        # d_out = train_target.shape[1]

        # built Win1, Win2
        if self.Win1 is None or self.Win2 is None:
            np.random.seed(self.seed+100)
            # build V1, V2 in shape [N, d_in]
            N = self.reservoir_size
            # V1 = (np.random.rand(N, d_in)-0.5)*2.0*self.input_scale
            # V2 = (np.random.rand(N, d_in)-0.5)*2.0*self.input_scale

            sign_V1 = np.random.choice([-1, 1], size=(N, d_in))
            sign_V2 = np.random.choice([-1, 1], size=(N, d_in))

            v1, v2 = self.v1, self.v2  # fixed values for V1, V2

            V1 = v1 * sign_V1 * self.input_scale
            V2 = v2 * sign_V2 * self.input_scale

            # eq(10): Win1= V1 - V2, Win2= V1 + V2
            self.Win1 = V1 - V2
            self.Win2 = V1 + V2

        # define x1, x2
        self._init_substates()

        # gather states
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]  # shape => [T-discard, d_out]

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        # Solve ridge
        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        # W_out => shape [d_out, 2N+1]
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        """
        Fully autoregressive: 
          We do not use teacher forcing, 
          we feed the model's last output as the next input 
        Typically, for MCI-ESN the paper does input(t+1) in R^d. 
        We do the test_input
        For multi-step chaotic forecast, we feed the model's output as input? 
        That means the system dimension d_in must match d_out. 
        """
        preds = []
        # re-init states
        #self._init_substates()

        # we assume initial_input => shape (d_in,)
        current_in = np.array(initial_input)

        for _ in range(n_steps):
            self._update(current_in)
            # read out
            combined = self._combine_state()
            big_x = augment_state_with_squares(combined)
            out = self.W_out @ big_x  # shape => (d_out,)

            preds.append(out)
            current_in = out  # feed output back as next input

        return np.array(preds)
        
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            combined = self._combine_state()
            x_aug = augment_state_with_squares(combined)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)

In [9]:
class ESN3D:
    """
    Dense random ESN for 3D->3D single-step.
    Teacher forcing for training, autoregressive for testing.
    """
    def __init__(self,
                 reservoir_size=300,
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        np.random.seed(self.seed)
        W = np.random.randn(reservoir_size, reservoir_size)*0.1
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W

        np.random.seed(self.seed+1)
        self.W_in = (np.random.rand(reservoir_size,3) - 0.5)*2.0*self.input_scale
        # self.W_in = np.random.uniform(-self.input_scale, self.input_scale, (reservoir_size, 3))

        self.W_out = None
        self.x = np.zeros(reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        # X_aug = np.hstack([states_use, np.ones((states_use.shape[0],1))])

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            # x_aug = np.concatenate([self.x, [1.0]])
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)

In [10]:
class SparseESN3D:
    """
    Sparse random ESN for 3D->3D single-step,
    teacher forcing for training, autoregressive for testing.
    """
    def __init__(self,
                 reservoir_size=300,
                 spectral_radius=0.95,
                 connectivity=0.05,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.connectivity = connectivity
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        np.random.seed(self.seed)
        W_full = np.random.randn(reservoir_size, reservoir_size)*0.1
        mask = (np.random.rand(reservoir_size, reservoir_size) < self.connectivity)
        W = W_full * mask
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W

        np.random.seed(self.seed+1)
        self.W_in = (np.random.rand(reservoir_size,3) - 0.5)*2.0*self.input_scale

        self.W_out = None
        self.x = np.zeros(reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        # X_aug = np.hstack([states_use, np.ones((states_use.shape[0],1))])

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            # x_aug = np.concatenate([self.x, [1.0]])
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)

In [11]:
class CR3D:
    """
    Cycle (ring) reservoir for 3D->3D single-step,
    teacher forcing for training, autoregressive for testing.
    """
    def __init__(self,
                 reservoir_size=300,
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        self.reservoir_size = reservoir_size
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        np.random.seed(self.seed)
        W = np.zeros((reservoir_size, reservoir_size))
        for i in range(reservoir_size):
            j = (i+1) % reservoir_size
            W[i, j] = 1.0
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W
        
        np.random.seed(self.seed+1)
        self.W_in = (np.random.rand(reservoir_size,3) - 0.5)*2.0*self.input_scale

        self.W_out = None
        self.x = np.zeros(reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        # X_aug = np.hstack([states_use, np.ones((states_use.shape[0],1))])

        # polynomial readout
        X_list = []
        for s in states_use:
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)  # shape => [T-discard, 2N+1]

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            # x_aug = np.concatenate([self.x, [1.0]])
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)

In [12]:
def scale_spectral_radius(W, target_radius=0.95):
    """
    Scales a matrix W so that its largest eigenvalue magnitude = target_radius.
    """
    eigvals = np.linalg.eigvals(W)
    radius = np.max(np.abs(eigvals))
    if radius == 0:
        return W
    return (W / radius) * target_radius

def augment_state_with_squares(x):
    """
    Given state vector x in R^N, return [ x, x^2, 1 ] in R^(2N+1).
    We'll use this for both training and prediction.
    """
    x_sq = x**2
    return np.concatenate([x, x_sq, [1.0]])  # shape: 2N+1

class HFRRes3D:
    """
    Hierarchical Fractal Reservoir (HFR) for 3D chaotic systems.
    
    This novel reservoir architecture partitions the chaotic attractor at multiple
    hierarchical scales, combining them in a fractal-like adjacency structure.
    The method is model-free, relying solely on the observed trajectory in R^3,
    and does not require knowledge of any system parameters such as sigma, rho, beta
    for Lorenz63. 
    
    Key Idea:
     1) Define multiple 'scales' of partition of the data's bounding region.
     2) Each scale is subdivided into a certain number of cells (regions).
     3) Each cell at level l has links to both:
        - other cells at the same level (horizontal adjacency),
        - 'child' cells at the finer level l+1 (vertical adjacency).
     4) We gather all cells across levels => a multi-level fractal graph => adjacency => W.
     5) We build a typical ESN from this adjacency, feed data with W_in, run leaky tanh updates,
        then do a polynomial readout for 3D next-step prediction.

    This approach is suitable for chaotic systems whose attractors often exhibit fractal
    self-similarity, thus capturing multi-scale structures in a single reservoir.
    """

    def __init__(self,
                 n_levels=3,             # number of hierarchical levels
                 cells_per_level=None,   # list of number of cells at each level, e.g. [8, 32, 128]
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        """
        Parameters
        ----------
        n_levels       : int, number of hierarchical scales
        cells_per_level: list[int], the number of partitions/cells at each level
                         if None, we auto-generate e.g. 2^(level+2)
        spectral_radius: final scaling for adjacency
        input_scale    : random input scale W_in
        leaking_rate   : ESN leaky alpha
        ridge_alpha    : readout ridge penalty
        seed           : random seed
        """
        self.n_levels        = n_levels
        self.cells_per_level = cells_per_level
        self.spectral_radius = spectral_radius
        self.input_scale     = input_scale
        self.leaking_rate    = leaking_rate
        self.ridge_alpha     = ridge_alpha
        self.seed            = seed

        if self.cells_per_level is None:
            # default scheme e.g. 8, 16, 32 for 3 levels
            self.cells_per_level = [8*(2**i) for i in range(n_levels)]

        # We'll store adjacency W, input W_in, readout W_out, reservoir state x
        self.W     = None
        self.W_in  = None
        self.W_out = None
        self.x     = None
        self.n_levels = len(self.cells_per_level)

        # We'll define a total number of nodes = sum(cells_per_level)
        self.n_nodes = sum(self.cells_per_level)

    def _build_partitions(self, data_3d):
        """
        Build hierarchical partitions for each level.
        We'll store the bounding box for data_3d, then for each level l in [0..n_levels-1]
        run e.g. k-means with K = cells_per_level[l], each point gets a label => we track transitions.

        Return: 
          partitions => list of arrays, partitions[l] => shape (N, ) cluster assignment in [0..cells_per_level[l]-1]
        """
        from sklearn.cluster import KMeans
        N = len(data_3d)
        partitions = []

        for level in range(self.n_levels):
            k = self.cells_per_level[level]
            # cluster
            kmeans = KMeans(n_clusters=k, random_state=self.seed+10*level, n_init='auto')
            kmeans.fit(data_3d)
            labels = kmeans.predict(data_3d)
            partitions.append(labels)

        return partitions

    def _build_hierarchical_adjacency(self, data_3d):
        """
        Build a block adjacency with cross-level links, then scale spectral radius.
        Steps:
          1) Build partitions for each level => partitions[l] in [0..cells_per_level[l]-1]
          2) For each level l, build a transition matrix T_l of shape (cells_per_level[l], cells_per_level[l]).
          3) Link scale l to scale l+1 by figuring out which cluster i at scale l maps to which cluster j at scale l+1
             for each sample t => link i-> j if data_3d[t] is in i at scale l and j at scale l+1.
          4) Combine all transitions in one big adjacency W in R^(n_nodes x n_nodes).
          5) row-normalize W => scale largest eigenvalue => spectral_radius
        """
        partitions = self._build_partitions(data_3d)
        N = len(data_3d)

        # offsets for each level => to index big W
        offsets = []
        running = 0
        for level in range(self.n_levels):
            offsets.append(running)
            running += self.cells_per_level[level]

        # total nodes
        n_tot = self.n_nodes
        # initialize adjacency
        A = np.zeros((n_tot, n_tot))

        # 1) horizontal adjacency in each level
        for level in range(self.n_levels):
            k = self.cells_per_level[level]
            labels = partitions[level]
            # T_l => shape (k, k)
            T_l = np.zeros((k, k))
            for t in range(N-1):
                i = labels[t]
                j = labels[t+1]
                T_l[i,j]+=1
            # row normalize
            row_sum = T_l.sum(axis=1, keepdims=True)
            row_sum[row_sum==0.0] = 1.0
            T_l /= row_sum
            # place T_l into big A
            off = offsets[level]
            A[off:off+k, off:off+k] = T_l

        # 2) vertical adjacency between scale l and l+1
        for level in range(self.n_levels-1):
            k_l   = self.cells_per_level[level]
            k_lp1 = self.cells_per_level[level+1]
            labels_l   = partitions[level]
            labels_lp1 = partitions[level+1]
            # we define adjacency from i in [0..k_l-1] to j in [0..k_lp1-1] if the same sample t belongs to i at level l and j at l+1
            # Count how many times
            Xvert1 = np.zeros((k_l, k_lp1))
            for t in range(N):
                i = labels_l[t]
                j = labels_lp1[t]
                Xvert1[i,j]+=1
            # row normalize
            row_sum = Xvert1.sum(axis=1, keepdims=True)
            row_sum[row_sum==0.0] = 1.0
            Xvert = Xvert1/row_sum
            # place in big A
            off_l   = offsets[level]
            off_lp1 = offsets[level+1]
            A[off_l:off_l+k_l, off_lp1:off_lp1+k_lp1] = Xvert
            # tentative idea, we could also define adjacency from l+1 -> l (parent link), if desired
            # we do the same for the 'child -> parent' link or skip it if we only want forward adjacency
            # For now, let's do symmetrical
            Yvert = Xvert1.T
            col_sum = Yvert.sum(axis=1, keepdims=True)
            col_sum[col_sum==0.0] = 1.0
            Yvert /= col_sum
            A[off_lp1:off_lp1+k_lp1, off_l:off_l+k_l] = Yvert

        # now we have a big adjacency => row normalize again, then scale spectral radius
        row_sum = A.sum(axis=1, keepdims=True)
        row_sum[row_sum==0.0] = 1.0
        A /= row_sum

        A = scale_spectral_radius(A, self.spectral_radius)
        return A

    def fit_readout(self, train_input, train_target, discard=100):
        """
        Main training routine:
          1) Build hierarchical adjacency from fractal partition => self.W
          2) define W_in => shape(n_nodes, 3)
          3) teacher forcing => polynomial readout => solve => self.W_out
        """
        np.random.seed(self.seed)
        # Build adjacency
        W_big = self._build_hierarchical_adjacency(train_input)
        self.W = W_big

        # define W_in => shape(n_nodes,3)
        self.n_nodes = W_big.shape[0]
        self.W_in = (np.random.rand(self.n_nodes,3)-0.5)*2.0*self.input_scale

        # define reservoir state
        self.x = np.zeros(self.n_nodes)

        # gather states => teacher forcing => polynomial => readout
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]
        X_list= []
        for s in states_use:
            X_list.append( augment_state_with_squares(s) )
        X_aug= np.array(X_list)

        reg= Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        self.W_out= reg.coef_

    def collect_states(self, inputs, discard=100):
        """
        Teacher forcing => feed real 3D => gather states => shape => [T-discard, n_nodes].
        returns (states_after_discard, states_discarded).
        """
        self.reset_state()
        states= []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states= np.array(states)
        return states[discard:], states[:discard]

    def reset_state(self):
        if self.x is not None:
            self.x.fill(0.0)

    def _update(self, u):
        """
        x(t+1)= (1-alpha)x(t)+ alpha tanh( W*x(t)+ W_in*u(t) ).
        """
        alpha= self.leaking_rate
        pre_acts= self.W@self.x + self.W_in@u
        x_new= np.tanh(pre_acts)
        self.x= (1.0- alpha)*self.x+ alpha*x_new

    def predict_autoregressive(self, initial_input, n_steps):
        """
        fully autonomous => feed last predicted => next input
        """
        preds= []
        #self.reset_state()
        current_in= np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            big_x= augment_state_with_squares(self.x)
            out= self.W_out@big_x
            preds.append(out)
            current_in= out
        return np.array(preds)
    
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


In [13]:
class CRJ3D:
    """
    Cycle Reservoir with Jumps (CRJ) for 3D->3D single-step tasks.
    We form a ring adjacency with an extra 'jump' edge in each row.
    This can help capture multiple timescales or delayed memory
    while retaining the easy ring structure.

    The adjacency is built as follows (reservoir_size = mod N):
      For each i in [0..N-1]:
        W[i, (i+1) % mod N] = 1.0
        W[i, (i+jump) % mod N] = 1.0
    Then we scale by 'spectral_radius.' We do an ESN update
    with readout [ x, x^2, 1 ] -> next step in R^3.
    """

    def __init__(self,
                 reservoir_size=300,
                 jump=10,                # offset for the jump
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=42):
        """
        reservoir_size: how many nodes in the ring
        jump            : the offset for the 2nd connection from node i
        spectral_radius : scale adjacency
        input_scale     : scale factor for W_in
        leaking_rate    : ESN 'alpha'
        ridge_alpha     : ridge penalty for readout
        seed            : random seed
        """
        self.reservoir_size = reservoir_size
        self.jump = jump
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        # build adjacency
        np.random.seed(self.seed)
        W = np.zeros((reservoir_size, reservoir_size))
        for i in range(reservoir_size):
            # cycle edge: i -> (i+1)%N
            W[i, (i+1) % reservoir_size] = 1.0
            # jump edge: i -> (i+jump)%N
            W[i, (i + self.jump) % reservoir_size] = 1.0

        # scale spectral radius
        W = scale_spectral_radius(W, self.spectral_radius)
        self.W = W

        # input weights => shape [N,3]
        np.random.seed(self.seed+100)
        W_in = (np.random.rand(reservoir_size, 3) - 0.5)*2.0*self.input_scale
        self.W_in = W_in

        # readout
        self.W_out = None
        self.x = np.zeros(self.reservoir_size)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        """
        Single-step ESN update:
          x(t+1) = (1-alpha)*x(t) + alpha*tanh( W x(t) + W_in u(t) )
        """
        pre_activation = self.W @ self.x + self.W_in @ u
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha)*self.x + alpha*x_new

    def collect_states(self, inputs, discard=100):
        """
        Teacher forcing => feed the real 3D inputs => gather states.
        Return (states_after_discard, states_discarded).
        """
        self.reset_state()
        states = []
        for val in inputs:
            self._update(val)
            states.append(self.x.copy())
        states = np.array(states)
        return states[discard:], states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        """
        gather states => polynomial readout => solve ridge
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        target_use = train_target[discard:]
        X_list = []
        for s in states_use:
            # polynomial expansion => [ x, x^2, 1 ]
            X_list.append(augment_state_with_squares(s))
        X_aug = np.array(X_list)

        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, target_use)
        self.W_out = reg.coef_  # shape => (3, 2N+1)

    def predict_autoregressive(self, initial_input, n_steps):
        """
        fully autoregressive => feed last output => next input
        """
        preds = []
        #self.reset_state()
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            big_x = augment_state_with_squares(self.x)
            out = self.W_out @ big_x  # shape => (3,)
            preds.append(out)
            current_in = out
        return np.array(preds)
        
    def predict_open_loop(self, test_input):
        preds = []
        for true_input in test_input:
            self._update(true_input)
            x_aug = augment_state_with_squares(self.x)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)

In [ ]:
# --- Revised segregated IO reservoir class ---
class SW3DSegregated:
    """
    Segregated IO small-world reservoir with spatial embedding.
    
    The reservoir is partitioned into three segments:
      - Input group: nodes that receive external input.
      - Core group: intermediate hidden nodes.
      - Output group: nodes from which the readout is taken.
    
    Total nodes = 500:
      - 100 Input nodes are clustered (angles in [-delta, delta])
      - 100 Output nodes are clustered (angles in [pi-delta, pi+delta])
      - 300 Core nodes are distributed evenly over the remaining circle.
    
    Connectivity is generated over the sorted order (by angular position) using
    a regular ring (nearest neighbor) scheme with random rewiring (Watts–Strogatz style).
    """
    def __init__(self,
                 reservoir_size=500,
                 edges_per_node=6,            # Must be even (here 6: 3 forward, 3 backward)
                 input_reservoir_size=100,      # Number of input nodes
                 output_reservoir_size=100,     # Number of output nodes
                 rewiring_probability=0.1,
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=48):
        # Save parameters
        self.reservoir_size = reservoir_size
        self.edges_per_node = edges_per_node
        self.input_res_size = input_reservoir_size
        self.output_res_size = output_reservoir_size
        self.core_res_size = reservoir_size - (input_reservoir_size + output_reservoir_size)
        self.rewiring_probability = rewiring_probability
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed
        
        np.random.seed(self.seed+1)
        
        # --- Assign spatial positions to nodes (angular embedding on a circle) ---
        # Define width for the input/output clusters.
        delta = np.pi / 20  # small angular window (≈9°)
        n_in = self.input_res_size
        n_out = self.output_res_size
        n_core = self.core_res_size
        
        # Input nodes: angles uniformly in [-delta, delta]
        input_positions = np.random.uniform(-delta, delta, size=n_in)
        # Output nodes: angles uniformly in [pi-delta, pi+delta]
        output_positions = np.random.uniform(np.pi - delta, np.pi + delta, size=n_out)
        
        # Core nodes: distribute them evenly over the remaining parts of the circle.
        # For clarity, we divide the remainder into two arcs.
        n_core1 = n_core // 2
        n_core2 = n_core - n_core1
        # Arc 1: from delta to (pi - delta)
        core_positions1 = np.linspace(delta, np.pi - delta, n_core1, endpoint=False)
        # Arc 2: from (pi + delta) to (2*pi - delta)
        core_positions2 = np.linspace(np.pi + delta, 2*np.pi - delta, n_core2, endpoint=False)
        core_positions = np.concatenate([core_positions1, core_positions2])
        
        # Concatenate all positions and assign labels:
        # 0 for input; 1 for core; 2 for output.
        self.positions = np.concatenate([input_positions, core_positions, output_positions])
        self.labels = np.concatenate([np.zeros(n_in, dtype=int),
                                       np.ones(n_core, dtype=int),
                                       2*np.ones(n_out, dtype=int)])
        
        # Sort nodes by angular position so they lie along a ring.
        sort_order = np.argsort(self.positions)
        self.positions = self.positions[sort_order]
        self.labels = self.labels[sort_order]
        self.all_indices = np.arange(self.reservoir_size)
        
        # Determine indices for each group after sorting.
        self.input_indices = np.where(self.labels == 0)[0]
        self.core_indices = np.where(self.labels == 1)[0]
        self.output_indices = np.where(self.labels == 2)[0]
        
        # --- Create input weight matrix (applied only to input nodes) ---
        # Assuming external input is 3-dimensional.
        self.W_in = (np.random.rand(len(self.input_indices), 3) - 0.5) * self.input_scale
        
        # Initialize output weight (readout) and reservoir state vector.
        self.W_out = None
        self.x = np.zeros(self.reservoir_size)
        
        # --- Build reservoir connectivity using the sorted (ring) order ---
        # We use a simple ring-based connection: for each node, connect to its
        # nearest neighbors (with offsets 1 to edges_per_node/2) in each direction.
        W = np.zeros((self.reservoir_size, self.reservoir_size))
        half_edges = self.edges_per_node // 2
        for i in range(self.reservoir_size):
            for offset in range(1, half_edges + 1):
                j = (i + offset) % self.reservoir_size
                k = (i - offset) % self.reservoir_size
                W[i, j] = 1.0
                W[i, k] = 1.0
        
        # --- Apply random rewiring (Watts–Strogatz) across all nodes ---
        for i in range(self.reservoir_size):
            current_neighbors = np.where(W[i] == 1.0)[0]
            for j in current_neighbors:
                if np.random.rand() < self.rewiring_probability:
                    W[i, j] = 0.0
                    # Ensure new connection does not duplicate an existing link
                    possible_nodes = list(set(self.all_indices) - {i} - set(np.where(W[i] == 1.0)[0]))
                    if possible_nodes:
                        new_j = np.random.choice(possible_nodes)
                        W[i, new_j] = 1.0
                        
        # Scale the reservoir connectivity to have the desired spectral radius.
        self.W = scale_spectral_radius(W, self.spectral_radius)

    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)

    def _update(self, u):
        """
        Update the reservoir state vector x.
          - The recurrent contribution is computed via W @ x.
          - The external input u (a 3D vector) is added only to input nodes (via W_in).
          - A tanh nonlinearity and leaky integration are applied.
        """
        recurrent = self.W @ self.x
        ext_input = np.zeros(self.reservoir_size)
        # Apply external input only to nodes in the input group.
        for idx, node in enumerate(self.input_indices):
            ext_input[node] = self.W_in[idx] @ u
        pre_activation = recurrent + ext_input
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha) * self.x + alpha * x_new

    def collect_states(self, inputs, discard=100):
        """
        Run the reservoir over an input sequence.
          - The external input drives only the input nodes.
          - States from the output nodes are recorded (after a washout period).
        Returns:
          - output_states: a T x (# output nodes) array (post-discard).
          - discard_states: states corresponding to the washout period.
        """
        self.reset_state()
        all_states = []
        for u in inputs:
            self._update(u)
            all_states.append(self.x.copy())
        all_states = np.array(all_states)
        output_states = all_states[:, self.output_indices]
        return output_states[discard:], output_states[:discard]

    def fit_readout(self, train_input, train_target, discard=100):
        """
        Train the linear readout on teacher-forced reservoir states.
        Only states from the output nodes (the segregated readout cluster)
        are used. A polynomial (squares) expansion and a bias term are
        applied before ridge regression.
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        X_list = [augment_state_with_squares(s) for s in states_use]
        X_aug = np.array(X_list)  # Shape: [T-discard, 2*len(output_indices) + 1]
        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_

    def predict_autoregressive(self, initial_input, n_steps):
        """
        Generate predictions in closed-loop (autoregressive) mode.
          - At each step, update the reservoir.
          - Compute the readout using only the output node states.
          - The prediction becomes the next input.
        """
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            x_out = self.x[self.output_indices]
            x_aug = augment_state_with_squares(x_out)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)
    
    def predict_open(self, test_input):
        preds = []
        for u in test_input:
            self._update(u)
            x_out = self.x[self.output_indices]
            x_aug = augment_state_with_squares(x_out)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)


# --- Random IO Reservoir Class ---
class SW3DRandom:
    """
    Random IO small-world reservoir for 3D->3D mapping.
    
    In this version:
      - Total reservoir nodes: reservoir_size (default 500)
      - Input nodes: a subset of reservoir nodes (default 100) chosen at random.
      - Output nodes: a disjoint subset of reservoir nodes (default 100) chosen at random.
      - The remaining nodes (core nodes) are the other 300 nodes.
    
    The recurrent connectivity is created using a ring-based nearest-neighbor scheme (with edges_per_node
    number of connections per node) and then rewired randomly with a given rewiring probability.
    
    The external input (assumed 3-dimensional) is injected only on the input nodes (via W_in),
    and the output is read only from the output nodes.
    """
    def __init__(self,
                 reservoir_size=500,
                 edges_per_node=6,              # Must be even (here: 3 neighbors forward, 3 backward)
                 input_reservoir_size=100,        # Number of input nodes
                 output_reservoir_size=100,       # Number of output nodes
                 rewiring_probability=0.1,
                 spectral_radius=0.95,
                 input_scale=1.0,
                 leaking_rate=1.0,
                 ridge_alpha=1e-6,
                 seed=48):
        # Save parameters
        self.reservoir_size = reservoir_size
        self.edges_per_node = edges_per_node
        self.input_res_size = input_reservoir_size
        self.output_res_size = output_reservoir_size
        self.core_res_size = reservoir_size - (input_reservoir_size + output_reservoir_size)
        self.rewiring_probability = rewiring_probability
        self.spectral_radius = spectral_radius
        self.input_scale = input_scale
        self.leaking_rate = leaking_rate
        self.ridge_alpha = ridge_alpha
        self.seed = seed

        np.random.seed(self.seed+1)
        total_indices = np.arange(self.reservoir_size)
        # Randomly choose indices for IO nodes without overlapping.
        all_io = np.random.choice(total_indices, size=(self.input_res_size + self.output_res_size), replace=False)
        np.random.shuffle(all_io)
        self.input_indices = all_io[:self.input_res_size]
        self.output_indices = all_io[self.input_res_size:self.input_res_size + self.output_res_size]
        # The core indices are the remaining ones
        self.core_indices = np.array(list(set(total_indices) - set(self.input_indices) - set(self.output_indices)))
        
        # --- Create Input Weight Matrix ---
        # Only input nodes receive external input; input is assumed 3-dimensional.
        self.W_in = (np.random.rand(len(self.input_indices), 3) - 0.5) * self.input_scale

        # Initialize output weight (readout) and reservoir state vector.
        self.W_out = None
        self.x = np.zeros(self.reservoir_size)
        
        # --- Build reservoir connectivity ---
        # Use a ring-based connectivity in the natural (index) order.
        # Each node i will be connected to its nearest half_edges neighbors in both directions.
        W = np.zeros((self.reservoir_size, self.reservoir_size))
        half_edges = self.edges_per_node // 2
        for i in range(self.reservoir_size):
            for offset in range(1, half_edges + 1):
                j = (i + offset) % self.reservoir_size
                k = (i - offset) % self.reservoir_size
                W[i, j] = 1.0
                W[i, k] = 1.0

        # --- Random rewiring (Watts-Strogatz style) ---
        for i in range(self.reservoir_size):
            current_neighbors = np.where(W[i] == 1.0)[0]
            for j in current_neighbors:
                if np.random.rand() < self.rewiring_probability:
                    W[i, j] = 0.0
                    possible_nodes = list(set(range(self.reservoir_size)) - {i} - set(np.where(W[i] == 1.0)[0]))
                    if possible_nodes:
                        new_j = np.random.choice(possible_nodes)
                        W[i, new_j] = 1.0

        # Scale the reservoir weight matrix to have the desired spectral radius.
        self.W = scale_spectral_radius(W, self.spectral_radius)
    
    def reset_state(self):
        self.x = np.zeros(self.reservoir_size)
    
    def _update(self, u):
        """
        Update the reservoir state.
          - The recurrent input is W @ x.
          - External input u (3D) is added only at input nodes.
          - A tanh nonlinearity and leaky integration update the state.
        """
        recurrent = self.W @ self.x
        ext_input = np.zeros(self.reservoir_size)
        for idx, node in enumerate(self.input_indices):
            ext_input[node] = self.W_in[idx] @ u
        pre_activation = recurrent + ext_input
        x_new = np.tanh(pre_activation)
        alpha = self.leaking_rate
        self.x = (1.0 - alpha) * self.x + alpha * x_new
    
    def collect_states(self, inputs, discard=100):
        """
        Drive the reservoir with a sequence of inputs and record the states.
        Only the states of the output nodes are kept for the readout.
        """
        self.reset_state()
        all_states = []
        for u in inputs:
            self._update(u)
            all_states.append(self.x.copy())
        all_states = np.array(all_states)
        output_states = all_states[:, self.output_indices]
        return output_states[discard:], all_states[:discard]
    
    def fit_readout(self, train_input, train_target, discard=100):
        """
        Train the readout weights from the reservoir.
          - The readout is based on states from output nodes.
          - Each state is augmented (with squared values and bias) before Ridge regression.
        """
        states_use, _ = self.collect_states(train_input, discard=discard)
        targets_use = train_target[discard:]
        X_list = [augment_state_with_squares(s) for s in states_use]
        X_aug = np.array(X_list)
        reg = Ridge(alpha=self.ridge_alpha, fit_intercept=False)
        reg.fit(X_aug, targets_use)
        self.W_out = reg.coef_
    
    def predict_autoregressive(self, initial_input, n_steps):
        """
        Generate predictions in an autoregressive manner.
          - At each step, the reservoir is updated with the current input.
          - The output is computed solely from the output nodes.
          - The predicted output is used as the next input.
        """
        preds = []
        current_in = np.array(initial_input)
        for _ in range(n_steps):
            self._update(current_in)
            x_out = self.x[self.output_indices]
            x_aug = augment_state_with_squares(x_out)
            out = self.W_out @ x_aug
            preds.append(out)
            current_in = out
        return np.array(preds)
    
    def predict_open(self, test_input):
        preds = []
        for u in test_input:
            self._update(u)
            x_out = self.x[self.output_indices]
            x_aug = augment_state_with_squares(x_out)
            out = self.W_out @ x_aug
            preds.append(out)
        return np.array(preds)

Systems

In [ ]:
# def lorenz_deriv(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
#     x, y, z = state
#     dxdt = sigma * (y - x)
#     dydt = x*(rho - z) - y
#     dzdt = x*y - beta*z
#     return [dxdt, dydt, dzdt]

# def generate_lorenz_data(
#     initial_state=[1.0, 1.0, 1.0],
#     tmax=25.0,
#     dt=0.01,
#     sigma=10.0,
#     rho=28.0,
#     beta=8.0/3.0
# ):
#     num_steps = int(tmax / dt) + 1 # +1 to include t=0
#     t_vals = np.linspace(0, tmax, num_steps)
#     sol = odeint(lorenz_deriv, initial_state, t_vals, args=(sigma, rho, beta))
#     return t_vals, sol

def report_vpt(name, preds, test_target, time_test, dt):
    T_VPT, T_lambda, ratio = compute_valid_prediction_time(
        test_target, preds, time_test, threshold=0.4, lambda_max=0.9, dt=dt
    )
    # print(f"{name:20s} => T_VPT={T_VPT:.3f},  T_lambda={T_lambda:.3f}, ratio={ratio:.3f}")
    return T_VPT, T_lambda, ratio

# # 1) Generate Lorenz data
# tmax = 250
# dt   = 0.02
# t_vals, lorenz_traj = generate_lorenz_data(
#     initial_state=[1.0,1.0,1.0],
#     tmax=tmax,
#     dt=dt
# )

# washout = 2000
# t_vals = t_vals[washout:]
# lorenz_traj = lorenz_traj[washout:]

# # normalize
# scaler = StandardScaler()
# scaler.fit(lorenz_traj)
# lorenz_traj = scaler.transform(lorenz_traj)

# T_data = len(lorenz_traj)
# print(f"Data length: {T_data}, from t=0..{tmax} with dt={dt}.")

n_test_steps = 2100

# # train/test split
# train_frac = 0.8
# train_end = int(train_frac*(T_data-1))
# train_input  = lorenz_traj[:train_end]
# train_target = lorenz_traj[1:train_end+1]
# test_input   = lorenz_traj[train_end:train_end+n_test_steps]
# test_target  = lorenz_traj[train_end+1:train_end+n_test_steps+1]
# print(f"Train size: {len(train_input)}  Test size: {len(test_input)}")

# initial_in = test_input[0]

Data length: 10501, from t=0..250 with dt=0.02.
Train size: 8400  Test size: 2100


In [ ]:
def lorenz_derivatives(state, t, sigma=10.0, rho=28.0, beta=8.0/3.0):
    """Compute time derivatives [dx/dt, dy/dt, dz/dt] for the Lorenz system."""
    x, y, z = state
    dxdt = sigma * (y - x)
    dydt = x * (rho - z) - y
    dzdt = x * y - beta * z
    return [dxdt, dydt, dzdt]

def generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0],
    tmax=25.0,
    dt=0.01,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
):
    """
    Numerically integrate Lorenz equations x'(t), y'(t), z'(t) using odeint.
    Returns: 
       t_vals: array of time points
       sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
    """
    num_steps = int(tmax / dt)
    t_vals = np.linspace(0, tmax, num_steps)
    sol = odeint(lorenz_derivatives, initial_state, t_vals, args=(sigma, rho, beta))
    return t_vals, sol

tmax = 150.0
dt = 0.02
lorenz_t_vals, lorenz_trajectory = generate_lorenz_data(
    initial_state=[1.0, 1.0, 1.0], 
    tmax=tmax,
    dt=dt,
    sigma=10.0,
    rho=28.0,
    beta=8.0/3.0
)

# Discard first 2,000 points as washout
washout = 2000
lorenz_t_vals = lorenz_t_vals[washout:]
lorenz_trajectory = lorenz_trajectory[washout:]

# Prepare single-step data: input(t) = [x(t), y(t), z(t)], target(t)=[x(t+1), y(t+1), z(t+1)]
inputs = lorenz_trajectory[:-1]
targets = lorenz_trajectory[1:]

data_size = len(lorenz_trajectory)-1
train_size = 4500
train_input = inputs[:train_size]
train_target = targets[:train_size]
test_input = inputs[train_size:]
test_target = targets[train_size:]
test_size = len(test_input)

# === Normalize to [-1, 1] using min-max from training data ===
data_min = train_input.min(axis=0)
data_max = train_input.max(axis=0)

def normalize(x):
    return 2.0 * (x - data_min) / (data_max - data_min) - 1.0

def denormalize(x_norm):
    return 0.5 * (x_norm + 1.0) * (data_max - data_min) + data_min

# Apply normalization
train_input = normalize(train_input)
train_target = normalize(train_target)
test_input = normalize(test_input)
test_target = normalize(test_target)

print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}")

initial_in = test_input[0]

Total samples: 5499, train size: 4500, test size: 999


In [ ]:
# def rossler_derivatives(state, t, a=0.2, b=0.2, c=5.7):
#     """Compute time derivatives [dx/dt, dy/dt, dz/dt] for the Rössler system."""
#     x, y, z = state
#     dxdt = -y - z
#     dydt = x + a * y
#     dzdt = b + z * (x - c)
#     return [dxdt, dydt, dzdt]

In [ ]:
# def generate_rossler_data(
#     initial_state=[1.0, 0.0, 0.0],
#     tmax=25.0,
#     dt=0.01,
#     a=0.2,
#     b=0.2,
#     c=5.7
# ):
#     """
#     Numerically integrate Rössler equations x'(t), y'(t), z'(t) using odeint.
#     Returns: 
#        t_vals: array of time points
#        sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
#     """
#     num_steps = int(tmax / dt)
#     t_vals = np.linspace(0, tmax, num_steps)
#     sol = odeint(rossler_derivatives, initial_state, t_vals, args=(a, b, c))
#     return t_vals, sol

In [ ]:
# tmax = 280.0
# dt = 0.02
# rossler_t_vals, rossler_trajectory = generate_rossler_data(
#     initial_state=[1.0, 0.0, 0.0],
#     tmax=tmax,
#     dt=dt,
#     a=0.2,
#     b=0.2,
#     c=5.7
# )

# # Discard first 2,000 points as washout
# washout = 2000
# rossler_t_vals = rossler_t_vals[washout:]
# rossler_trajectory = rossler_trajectory[washout:]

# # Prepare single-step data: input(t) = [x(t), y(t), z(t)], target(t)=[x(t+1), y(t+1), z(t+1)]
# inputs = rossler_trajectory[:-1]
# targets = rossler_trajectory[1:]

# data_size = len(rossler_trajectory)-1
# train_size = 4500
# train_input = inputs[:train_size]
# train_target = targets[:train_size]
# test_input = inputs[train_size:]
# test_target = targets[train_size:]
# test_size = len(test_input)
# print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

# initial_in = test_input[0]

In [ ]:
# def chen_derivatives(state, t, a=35.0, b=3.0, c=28.0):
#     """
#     Computes time derivatives [dx/dt, dy/dt, dz/dt] for the Chen system.
#     """
#     x, y, z = state
#     dxdt = a*(y - x)
#     dydt = (c - a)*x + c*y - x*z
#     dzdt = x*y - b*z
#     return [dxdt, dydt, dzdt]

# def generate_chen_data(
#     initial_state=[1.0, 1.0, 1.0],
#     tmax=50.0,
#     dt=0.01,
#     a=35.0,
#     b=3.0,
#     c=28.0
# ):
#     """
#     Numerically integrate Chen equations x'(t), y'(t), z'(t) using odeint.
#     Returns: 
#        t_vals: array of time points
#        sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
#     """
#     num_steps = int(tmax / dt)
#     t_vals = np.linspace(0, tmax, num_steps)
#     sol = odeint(chen_derivatives, initial_state, t_vals, args=(a, b, c))
#     return t_vals, sol

# tmax = 150.0
# dt = 0.02
# chen_t_vals, chen_trajectory = generate_chen_data(
#     initial_state=[1.0, 1.0, 1.0],
#     tmax=tmax,
#     dt=dt,
#     a=35.0,
#     b=3.0,
#     c=28.0
# )

# # Discard first 2,000 points as washout
# washout = 2000
# chen_t_vals = chen_t_vals[washout:]
# chen_trajectory = chen_trajectory[washout:]

# # Prepare single-step data: input(t) = [x(t), y(t), z(t)], target(t)=[x(t+1), y(t+1), z(t+1)]
# inputs = chen_trajectory[:-1]
# targets = chen_trajectory[1:]

# data_size = len(chen_trajectory)-1
# train_size = 4500
# train_input = inputs[:train_size]
# train_target = targets[:train_size]
# test_input = inputs[train_size:]
# test_target = targets[train_size:]
# test_size = len(test_input)
# print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

# initial_in = test_input[0]

In [ ]:
# def chua_derivatives(state, t, alpha, beta, m0, m1):
#     """
#     Computes time derivatives [dx/dt, dy/dt, dz/dt] for the Chua system.
#     """
#     x, y, z = state

#     # Nonlinear function h(x)
#     h = m1 * x + 0.5 * (m0 - m1) * (np.abs(x + 1) - np.abs(x - 1))

#     dxdt = alpha * (y - x - h)
#     dydt = x - y + z
#     dzdt = -beta * y

#     return [dxdt, dydt, dzdt]

# def generate_chua_data(
#     initial_state=[0.1, 0.0, 0.0],
#     tmax=100.0,
#     dt=0.01,
#     alpha=15.6,
#     beta=28.0,
#     m0=-1.143,
#     m1=-0.714
# ):
#     """
#     Numerically integrate Chua's circuit equations.
#     Returns:
#         t_vals: array of time points
#         sol   : array shape [num_steps, 3] of [x(t), y(t), z(t)]
#     """
#     num_steps = int(tmax / dt)
#     t_vals = np.linspace(0, tmax, num_steps)
#     sol = odeint(chua_derivatives, initial_state, t_vals, args=(alpha, beta, m0, m1))
#     return t_vals, sol

# tmax = 154.0
# dt = 0.02
# chua_t_vals, chua_trajectory = generate_chua_data(
#     initial_state=[0.2, 0.0, 0.0], 
#     tmax=tmax,
#     dt=dt,
#     alpha=15.6,
#     beta=28.0,
#     m0=-1.143,
#     m1=-0.714
# )

# # Discard first 2,000 points as washout
# washout = 2200
# chua_t_vals = chua_t_vals[washout:]
# chua_trajectory = chua_trajectory[washout:]

# # Prepare single-step data: input(t) = [x(t), y(t), z(t)], target(t)=[x(t+1), y(t+1), z(t+1)]
# inputs = chua_trajectory[:-1]
# targets = chua_trajectory[1:]

# data_size = len(chua_trajectory)-1
# train_size = 4500
# train_input = inputs[:train_size]
# train_target = targets[:train_size]
# test_input = inputs[train_size:]
# test_target = targets[train_size:]
# test_size = len(test_input)
# print(f"Total samples: {data_size}, train size: {train_size}, test size: {test_size}") 

# initial_in = test_input[0]

Predictions

In [ ]:
# Define your parameter grids
param_grid_mci = {
    "reservoir_size": [300],
    "cycle_weight": [0.6],
    "connect_weight": [1],
    "input_scale": [0.5],
    "leaking_rate": [0.5],
    "ridge_alpha": [1.00E-07],
    "combine_factor": [0.3],
    "v1": [0.4],
    "v2": [0.4]
}

# Define your parameter grids
param_grid_esn = {
    "spectral_radius": [0.9],
    "input_scale": [0.5],
    "leaking_rate": [0.6],
    "ridge_alpha": [1e-7]
}

param_grid_cycle = {
    "spectral_radius": [0.8],
    "input_scale": [0.3],
    "leaking_rate": [0.7],
    "ridge_alpha": [1e-7]
}

param_grid_sparse = {
    "spectral_radius": [0.9],
    "input_scale": [0.2],
    "leaking_rate": [0.5],
    "ridge_alpha": [1e-7],
    "connectivity": [0.08]
}

cells_per_level_grid = {
    "cells_per_level": [[
        5,
        15,
        25,
        30,
        35,
        40,
        45,
        105
      ]],
    "spectral_radius": [0.9],
    "input_scale": [0.2],
    "leaking_rate": [0.9],
    "ridge_alpha": [1e-8],
}

param_grid_crj = {
    "spectral_radius": [0.92],
    "input_scale": [0.6],
    "leaking_rate": [0.4],
    "ridge_alpha": [1e-7],
    "jump": [20] 
}

param_grid_sws = {
    "edges_per_node": [2],
    "input_scale": [0.4],
    "rewiring_probability": [1],
    "spectral_radius": [0.8],
}

param_grid_swr = {
    "edges_per_node": [4],
    "input_scale": [0.2],
    "rewiring_probability": [0.2],
    "spectral_radius": [0.8],
}

# def run_grid_search(model_class, param_grid, model_name,
#                     output_path="grid_search_results.json"):
#     # Precompute param combinations
#     combos = list(itertools.product(*param_grid.values()))
#     param_keys = list(param_grid.keys())
#     print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

#     results = []
#     # tqdm adds a progress bar for better visualization
#     for comb in tqdm(combos, desc="Grid Search"):
#         params = dict(zip(param_keys, comb))
#         seed_scoresvpt = []
#         seed_scoresadev = []
#         seed_scoresnrmse = []
        
#         # Run all 20 seeds
#         for seed in range(1, 31):
#             model = model_class(**params, seed=seed)
#             model.fit_readout(train_input, train_target, discard=100)
#             preds = model.predict_autoregressive(initial_in, n_test_steps)
#             T_VPT_s, _, _ = report_vpt("model", preds,
#                                        test_target,
#                                        t_vals[train_end:train_end+n_test_steps],
#                                        dt)
#             adev = compute_attractor_deviation(preds, test_target)
#             nrmse = nrmse_dimwise(preds, test_target)
#             seed_scoresvpt.append(T_VPT_s)
#             seed_scoresadev.append(adev)
#             seed_scoresnrmse.append(nrmse)

#         mean_scorevpt = float(np.mean(seed_scoresvpt))
#         std_devvpt    = float(np.std(seed_scoresvpt))
#         mean_scoreadev = float(np.mean(seed_scoresadev))
#         std_devadev    = float(np.std(seed_scoresadev))
#         mean_scoresnrmse = float(np.mean(seed_scoresnrmse))
#         std_devsnrmse = float(np.std(seed_scoresnrmse))
#         # is_stable  = std_dev < 1.5
#         # status     = "Stable" if is_stable else "Unstable"
        
#         # print(f"Params: {params} → Avg T_VPT={mean_score:.3f}, "
#         #       f"Std Dev={std_dev:.3f} → {status}")

#         results.append({
#             "params":      params,
#             # "seed_scores": seed_scoresvpt,
#             "mean_T_VPT":  mean_scorevpt,
#             "std_dev":     std_devvpt,
#             "mean_adev":   mean_scoreadev,
#             "std_dev_adev": std_devadev,
#             "mean_nrmse": mean_scoresnrmse,
#             "std_dev_nrmse": std_devsnrmse,
#             "model_name":  model_name
#         })

#     # Save results
#     with open(output_path, "w") as f:
#         json.dump(results, f, indent=2)
#     print(f"\nAll results saved to `{output_path}`")
    
#     return results

In [17]:
PREDICTION_HORIZONS = [200, 400, 600, 800, 1000]

def run_grid_search(model_class, param_grid, model_name,
                    output_path="grid_search_results.json"):
    combos = list(itertools.product(*param_grid.values()))
    param_keys = list(param_grid.keys())
    print(f"\n== Initial grid search for {model_name} with {len(combos)} combinations ==")

    results = []
    horizon_nrmse_dict = {h: [] for h in PREDICTION_HORIZONS}

    for comb in tqdm(combos, desc="Grid Search"):
        params = dict(zip(param_keys, comb))

        # Store per-seed NRMSE for each horizon
        nrmse_across_seeds = {h: [] for h in PREDICTION_HORIZONS}

        for seed in range(1, 31):
            model = model_class(**params, seed=seed)
            model.fit_readout(train_input, train_target, discard=100)
            preds = model.predict_autoregressive(initial_in, n_test_steps)
            for h in PREDICTION_HORIZONS:
                nrmse_h = nrmse_dimwise(preds[:h], test_target[:h])
                nrmse_across_seeds[h].append(nrmse_h)

        model_horizon_stats = {}
        for h in PREDICTION_HORIZONS:
            mean_h = float(np.mean(nrmse_across_seeds[h]))
            std_h  = float(np.std(nrmse_across_seeds[h]))
            model_horizon_stats[h] = {"mean": mean_h, "std": std_h}
            horizon_nrmse_dict[h].append((model_name, mean_h))

        results.append({
            "params": params,
            "model_name": model_name,
            "nrmse_by_horizon": model_horizon_stats
        })

    # Save all results
    with open(output_path, "w") as f:
        json.dump(results, f, indent=2)
    print(f"\nAll results saved to `{output_path}`")

    return results, horizon_nrmse_dict


In [ ]:
mci     = run_grid_search(MCI3D, param_grid_mci, "MCI", 
                          output_path="grid_search_results_mci.json")
esn     = run_grid_search(ESN3D, param_grid_esn, "Dense ESN", 
                          output_path="grid_search_results_esn.json")
cr      = run_grid_search(CR3D, param_grid_cycle, "Cycle Res", 
                          output_path="grid_search_results_cr.json")
sparse  = run_grid_search(SparseESN3D, param_grid_sparse, "Sparse ESN",
                          output_path="grid_search_results_sparse.json")
crj     = run_grid_search(CRJ3D, param_grid_crj, "Cycle Res with Jumps", 
                      output_path="grid_search_results_crj.json")
sws     = run_grid_search(SW3DSegregated, param_grid_sws, "Small-World",
                      output_path="grid_search_results_sws.json")
swr     = run_grid_search(SW3DRandom, param_grid_swr, "Small-World Random",
                      output_path="grid_search_results_swr.json")
hfr     = run_grid_search(HFRRes3D, cells_per_level_grid, "HFR Res",
                      output_path="grid_search_results_hfr.json")


== Initial grid search for MCI with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [00:24<00:00, 24.51s/it]



All results saved to `grid_search_results_mci.json`

== Initial grid search for Dense ESN with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [00:11<00:00, 11.18s/it]



All results saved to `grid_search_results_esn.json`

== Initial grid search for Cycle Res with 1 combinations ==


...


All results saved to `grid_search_results_sws.json`

== Initial grid search for Small-World Random with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [01:06<00:00, 66.51s/it]



All results saved to `grid_search_results_swr.json`

== Initial grid search for HFR Res with 1 combinations ==


Grid Search: 100%|██████████| 1/1 [00:20<00:00, 20.50s/it]


All results saved to `grid_search_results_hfr.json`


In [ ]:
# def print_nrmse_table(all_results_dict):
#     print("\nNRMSE for Different Prediction Horizons:")
#     print("-" * 120)
#     print(f"{'Horizon':<10}", end="")
#     model_names = list(next(iter(all_results_dict.values())))
#     for name, _ in model_names:
#         print(f"{name:<16}", end="")
#     print("\n" + "-" * 120)

#     for h in PREDICTION_HORIZONS:
#         print(f"{h:<10}", end="")
#         for _, mean in all_results_dict[h]:
#             print(f"{mean:<16.7f}", end="")
#         print()



# # Combine into one dictionary
# all_results = {}
# for d in [mci_nrmse, esn_nrmse, cr_nrmse, sparse_nrmse,
#           crj_nrmse, sws_nrmse, swr_nrmse, hfr_nrmse]:
#     for h in d:
#         all_results.setdefault(h, []).extend(d[h])

# print_nrmse_table(all_results)
